In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

import pandas as pd
import csv

In [24]:
def extract_ship_info(imo_numbers):
    # Initialize the Chrome driver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)

    # Initialize an empty dataframe
    df = pd.DataFrame()

    for imo_number in imo_numbers:
        # Access the page
        url = f"https://www.balticshipping.com/vessel/imo/{imo_number}"
        driver.get(url)

        # Get the complete HTML content of the page
        html_content = driver.page_source

        # Create a BeautifulSoup object to parse the HTML
        soup = BeautifulSoup(html_content, "html.parser")

        # Find the div with the class "ship-info-container"
        ship_info_container = soup.find("div", {"class": "ship-info-container", "style": "position: relative;"})

        if ship_info_container:
            # Find the table within the div
            ship_info_table = ship_info_container.find("table", {"class": "table ship-info", "style": "min-height: 710px;"})

            if ship_info_table:
                # Extract the data from the table
                data = {}
                rows = ship_info_table.find_all("tr")
                for row in rows:
                    cells = row.find_all("td")
                    headers = row.find_all("th")
                    if headers and cells:
                        data[headers[0].text.strip()] = [cell.text.strip() for cell in cells]

                # Convert the data to a dataframe
                new_df = pd.DataFrame(data)

                # Concatenate the new dataframe with the main dataframe
                df = pd.concat([df, new_df], ignore_index=True)

            else:
                print(f"Could not find the ship information table for IMO {imo_number}")
        else:
            print(f"Could not find the ship information section for IMO {imo_number}")

    # Close the browser
    driver.quit()

    # Return the dataframe
    return df


In [25]:
# List of IMO numbers
imo_numbers = [9223887, 9660542, 8423155]
extract_ship_info(imo_numbers)



,IMO number,MMSI,Name of the ship,Former names,Vessel type,Operating status,Flag,Gross tonnage,Deadweight,Length,...,Engine type,Engine power,Year of build,Builder,Classification society,Home port,Owner,Manager,Description,Engine model
0,9223887,403544000,SAFWA,SAFWG ...,Crude oil tanker,Active,Saudi Arabia,159990 tons,303139 tons,333 m,...,B&W,32185 KW,2002,SAMSUNG SHIPBUILDING & HEAVY INDUSTRIES CO. LT...,AMERICAN BUREAU OF SHIPPING,DAMMAM,BAHRI SHIPMANAGEMENT,"MIDEAST SHIPMANAGEMENT - DUBAI, UNITED ARAB EM...",SAFWA is a Crude oil tanker built in 2002 by S...,NaN
1,9660542,403518000,BAHRI WAFI,APBI WAFI ...,Bulk carrier,Active,Saudi Arabia,43894 tons,81855 tons,229 m,...,Wartsila,9480 KW,2014,"OSHIMA SHIPBUILDING CO. LTD. - SAIKAI, JAPAN",LLOYD'S SHIPPING REGISTER,DAMMAM,BAHRI SHIPMANAGEMENT,"MIDEAST SHIPMANAGEMENT - DUBAI, UNITED ARAB EM...",BAHRI WAFI is a Bulk carrier built in 2014 by ...,6RT-flex58T
2,8423155,275460000,SALDUS,SALDUS ...,Fishing vessel,Active,Latvia,723 tons,414 tons,54 m,...,NaN,NaN,1985,"LENINSKA KUZNYA - KIEV, UKRAINE",INTERNATIONAL REGISTER OF SHIPPING,RIGA,NORTH STAR LTD,NORTH STAR LTD,SALDUS is a Fishing vessel built in 1985 by LE...,NaN
